In [14]:
import pandas as pd
import numpy as nmp
import wrangle 
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from pydataset import data

# Exercises

* Do your work for this exercise in a jupyter notebook named feature_engineering within the regression-exercises repo. Add, commit, and push your work.

* Load the tips dataset.

In [3]:
data('tips', show_doc=True)

tips

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Tipping data

### Description

One waiter recorded information about each tip he received over a period of a
few months working in one restaurant. He collected several variables:

### Usage

    data(tips)

### Format

A data frame with 244 rows and 7 variables

### Details

  * tip in dollars, 

  * bill in dollars, 

  * sex of the bill payer, 

  * whether there were smokers in the party, 

  * day of the week, 

  * time of day, 

  * size of the party. 

In all he recorded 244 tips. The data was reported in a collection of case
studies for business statistics (Bryant & Smith 1995).

### References

Bryant, P. G. and Smith, M (1995) _Practical Data Analysis: Case Studies in
Business Statistics_. Homewood, IL: Richard D. Irwin Publishing:




In [5]:
df = data('tips')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 15.2+ KB


In [19]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,6.147500


In [21]:
df = pd.get_dummies(df)


## Create a column named price_per_person. This should be the total bill divided by the party size.


In [22]:
df['price_per_person'] = df['total_bill'] / df['size']

In [23]:
# split data into train validate test
train_validate, test = train_test_split(df, test_size=.2, random_state=514)
train, validate = train_test_split(train_validate, test_size=.3, random_state=514)

In [24]:
# further breakdown for X_train, y_train .. etc
X_train = train.drop(columns='tip')
y_train = train.tip
X_validate = validate.drop(columns='tip')
y_validate = validate.tip
X_test = test.drop(columns='tip')
y_test = test.tip

## Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
* Day of the week, and size of the party

## Use select k best to select the top 2 features for predicting tip amount. What are they?


In [35]:
# initilize selector object
f_selector = SelectKBest(f_regression, k=2)

#fit object --> will find top 2 as requested
f_selector.fit(X_train, y_train)

# create mask
feature_mask = f_selector.get_support()

# use mask to show list of feature support
f_top_features = X_train.iloc[:,feature_mask].columns.tolist()

f_top_features

['total_bill', 'size']

## Use recursive feature elimination to select the top 2 features for tip amount. What are they?


In [31]:
#initialize  regression object
lm = LinearRegression()

# initilize RFE object with 2 features
rfe = RFE(lm, n_features_to_select=2)

#fit object onto data
rfe.fit(X_train, y_train)

#create boolean mask for columns model selects 
feature_mask = rfe.support_

# use mask to show list of selected features
rfe_top_features = X_train.iloc[:, feature_mask].columns.tolist()


In [32]:
rfe_top_features

['time_Dinner', 'time_Lunch']

## Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?
* Selecting based off of different criteria will give differet results in the first round of feature competition. If different features win out, then their competition in later selection rounds will differ. 
* The results have less chance of being similiar with lower k/n amounts. Increasing the size of k/n did not appear to increase changes of them being similiar 

Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.


In [33]:
def select_kbest(X, y, k): 
    # initilize selector object
    f_selector = SelectKBest(f_regression, k=k)

    #fit object --> will find top 2 as requested
    f_selector.fit(X, y)

    # create mask
    feature_mask = f_selector.get_support()

    # use mask to show list of feature support
    f_top_features = X.iloc[:,feature_mask].columns.tolist()

    return f_top_features

In [41]:
select_kbest(X_train, y_train, 4)

['total_bill', 'size', 'price_per_person', 'smoker_Yes']

## Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. 
Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [36]:
def rfe (X, y, n):

    #initialize  regression object
    lm = LinearRegression()

    # initilize RFE object with 2 features
    rfe = RFE(lm, n_features_to_select=n)

    #fit object onto data
    rfe.fit(X, y)

    #create boolean mask for columns model selects 
    feature_mask = rfe.support_

    # use mask to show list of selected features
    rfe_top_features = X.iloc[:, feature_mask].columns.tolist()

    return rfe_top_features

In [40]:
rfe(X_train, y_train, 4)

['sex_Female', 'day_Thur', 'time_Dinner', 'time_Lunch']

## Load the swiss dataset  
* Also split in sets for use. good for practice

In [42]:
swiss = data('swiss')

In [45]:
swiss.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Courtelary to Rive Gauche
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fertility         47 non-null     float64
 1   Agriculture       47 non-null     float64
 2   Examination       47 non-null     int64  
 3   Education         47 non-null     int64  
 4   Catholic          47 non-null     float64
 5   Infant.Mortality  47 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 2.6+ KB


In [46]:
# split data into train validate test
train_validate, test = train_test_split(swiss, test_size=.2, random_state=514)
train, validate = train_test_split(train_validate, test_size=.3, random_state=514)

In [47]:
# further breakdown for X_train, y_train .. etc
X_train = train.drop(columns='Fertility')
y_train = train.Fertility
X_validate = validate.drop(columns='Fertility')
y_validate = validate.Fertility
X_test = test.drop(columns='Fertility')
y_test = test.Fertility

### Use all the other features to predict Fertility. 

In [49]:
#initilize model and run with all features 
lm = LinearRegression()

In [50]:
#fit
lm.fit(X_train, y_train)

LinearRegression()

In [54]:
predictions = pd.DataFrame(index=X_validate.index)

In [55]:
predictions['actual'] = y_validate

In [56]:
predictions['lm - all features'] = lm.predict(X_validate)

### Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [59]:
# get select kbest features 
kbest_features = select_kbest(X_train, y_train, 3)

In [68]:
# train new model on features and then predict
lm_2 = LinearRegression()

lm_2.fit(X_train[kbest_features], y_train)

# add predictions 
predictions['lm_2 - kbest 3 feat'] = lm_2.predict(X_validate[kbest_features])

In [69]:
# train new model on features and then predict
lm_3 = LinearRegression()

# use rfe() to get 3 best features
rfe_features = rfe(X_train, y_train, 3)

#fit model on X_train, y_train
lm_3.fit(X_train[rfe_features], y_train)

# add predictions 
predictions['lm_3 - rfe 3 feat'] = lm_3.predict(X_validate[rfe_features])

In [70]:
predictions

,actual,lm - all features,lm_2 - kbest 3 feat,lm_3 - rfe 3 feat
Monthey,79.4,84.699884,79.796269,79.796269
Rive Droite,44.7,57.328461,61.307169,61.307169
Sierre,92.2,77.115653,75.074363,75.074363
St Maurice,65.0,73.761129,72.125819,72.125819
Herens,77.3,78.841834,77.907177,77.907177
Lausanne,55.7,57.870420,59.716674,59.716674
Porrentruy,76.1,95.231133,88.536713,88.536713
Franches-Mnt,92.5,88.722082,80.136157,80.136157
Martigwy,70.5,76.407780,74.460524,74.460524
Yverdon,65.4,74.440103,77.547135,77.547135
